<a href="https://colab.research.google.com/github/freedom73011/Python-01/blob/main/ch11%E8%A1%8C%E5%8B%95%E8%82%A1%E5%B8%82%E5%8D%B3%E6%99%82%E5%A0%B1%E5%83%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 台灣股市資訊模組：twstock

### 查詢歷史股票資料

In [ ]:
!pip install twstock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.4 MB/s eta 0:00:00


In [ ]:
import twstock

In [ ]:
stock = twstock.Stock('2317')

In [ ]:
print(stock.price)

[103.5, 103.5, 104.0, 103.5, 104.0, 104.5, 105.0, 104.5, 104.0, 104.0, 101.5, 101.0, 100.0, 100.5, 100.5, 100.0, 100.0, 98.6, 99.9, 100.5, 99.9, 101.0, 100.5, 102.0, 102.5, 102.5, 102.0, 102.5, 103.0, 102.0, 101.5]


In [ ]:
print(stock.price[-5:])

[102.0, 102.5, 103.0, 102.0, 101.5]


### 查詢即時股票資料

In [ ]:
real = twstock.realtime.get('2317')
real

{'timestamp': 1707114600.0,
 'info': {'code': '2317',
  'channel': '2317.tw',
  'name': '鴻海',
  'fullname': '鴻海精密工業股份有限公司',
  'time': '2024-02-05 06:30:00'},
 'realtime': {'latest_trade_price': '101.5000',
  'trade_volume': '4307',
  'accumulate_trade_volume': '17309',
  'best_bid_price': ['101.5000',
   '101.0000',
   '100.5000',
   '100.0000',
   '99.9000'],
  'best_bid_volume': ['854', '5502', '3282', '3512', '239'],
  'best_ask_price': ['102.0000',
   '102.5000',
   '103.0000',
   '103.5000',
   '104.0000'],
  'best_ask_volume': ['2062', '2603', '4806', '1763', '1803'],
  'open': '101.5000',
  'high': '102.0000',
  'low': '101.0000'},
 'success': True}

In [ ]:
real['realtime']['latest_trade_price']

'101.5000'

In [ ]:
import twstock

stock = twstock.Stock('2317')  #鴻海
print('近31個收盤價：')
print(stock.price)   #近31個收盤價
print('近6個收盤價：')
print(stock.price[-6:])   #近6日之收盤價
while True:
    real = twstock.realtime.get('2317')
    if real['success']:
        if real['realtime']['latest_trade_price'] != '-':
            print('即時股票資料：')
            print(real)  #即時資料
            print('目前股價：')
            print(real['realtime']['latest_trade_price']) #即時價格
            break
    else:
        print('錯誤：' + real['rtmessage'])


## 發送 LINE Notify 通知

In [ ]:
import requests

msg = '這是 LINE Notify 發送的訊息。'
token = 'REY8GjrzKlZtAY6pN05NTPaMDsGmyAjxHcpQb6fW2VO'  #權杖
headers = {
    "Authorization": "Bearer " + token,
    "Content-Type" : "application/x-www-form-urlencoded"
}
payload = {'message': msg}
notify = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
if notify.status_code == 200:
    print('發送 LINE Notify 成功！')
else:
    print('發送 LINE Notify 失敗！')

發送 LINE Notify 成功！


## 實戰：用LINE傳送即時股價

In [ ]:
import twstock
import time
import requests

def lineNotify(token, msg):
    headers = {
        "Authorization": "Bearer " + token,
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    payload = {'message': msg}
    notify = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return notify.status_code

def sendline(mode, realprice, counterLine, token):
    print('鴻海目前股價：' + str(realprice))
    if mode == 1:
        message = '現在鴻海股價為 ' + str(realprice) + '元，可以賣出股票了！'
    else:
        message = '現在鴻海股價為 ' + str(realprice) + '元，可以買入股票了！'
    code = lineNotify(token, message)
    if code == 200:
        counterLine = counterLine + 1
        print('第 ' + str(counterLine) + ' 次發送 LINE 訊息。')
    else:
        print('發送 LINE 訊息失敗！')
    return counterLine

token = 'REY8GjrzKlZtAY6pN05NTPaMDsGmyAjxHcpQb6fW2VO'  #權杖
counterLine = 0  #儲存發送次數
counterError = 0  #儲存錯誤次數

print('程式開始執行！')
while True:
    realdata = twstock.realtime.get('2317')  #即時資料
    if realdata['success']:
        realprice = realdata['realtime']['latest_trade_price']  #目前股價
        if realprice != '-':
            if float(realprice) >= 80:
                counterLine = sendline(1, realprice, counterLine, token)
            elif float(realprice) <= 60:
                counterLine = sendline(2, realprice, counterLine, token)
            if counterLine >= 3:  #最多發送3次就結束程式
                print('程式結束！')
                break
            for i in range(300):  #每5分鐘讀一次
                time.sleep(1)
    else:
        print('twstock 讀取錯誤，錯誤原因：' + realdata['rtmessage'])
        counterError = counterError + 1
        if counterError >= 3:  #最多錯誤3次
            print('程式結束！')
            break


程式開始執行！
鴻海目前股價：101.5000
第 1 次發送 LINE 訊息。
鴻海目前股價：101.5000
第 2 次發送 LINE 訊息。
鴻海目前股價：101.5000
第 3 次發送 LINE 訊息。
程式結束！
